In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.camera_utils import *

In [2]:
run_name = "RUN3"

spiegel_calib = ["spiegel.mp4", "spiegel1.mp4", "spiegel2.mp4"]
front_calib = ["front.mp4", "front1.mp4", "front2.mp4"]
stereo_calib = ["stereo.mp4", "stereo2.mp4", "stereo3.mp4", "stereo4.mp4", "stereo5.mp4", "stereo6.mp4", "stereo7.mp4"]

In [3]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (480, 240), 1: (480, 240)})
vL = videoLoader()


# Camera Positions

In [ ]:
vL.load_video(os.path.join(detection_dir, detection_files[-1]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
sC.set_anchor_point(frame, 0)
sC.set_anchor_point(frame, 1)
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


# Mirror Calibration

In [ ]:
frames = []
for file in spiegel_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
    
sC.calibrate(frames,0, rows=6, columns=8)

# Frontal Calibration

In [ ]:
frames = []
for file in front_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
    
sC.calibrate(frames,1, rows=6, columns=8)

# Stereo Calibration

In [4]:
frames = []

for file in stereo_calib:
    print(os.path.join(calibration_dir, file))
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100])
sC.load_from_yaml(f"{run_name}.yaml")
objectpoints, imgpoints_1, imgpoints_2, images = sC.stereo_calibrate(frames[:], rows=8, columns=10)

../videos/RUN3/calibration/stereo.mp4
../videos/RUN3/calibration/stereo2.mp4
../videos/RUN3/calibration/stereo3.mp4
../videos/RUN3/calibration/stereo4.mp4
../videos/RUN3/calibration/stereo5.mp4
../videos/RUN3/calibration/stereo6.mp4
../videos/RUN3/calibration/stereo7.mp4


QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread

Current line: [573, 209]
Current line: [573, 209, 556, 391]
Current line: [597, 394]
Current line: [597, 394, 615, 210]
Current line: [654, 211]
Current line: [654, 211, 638, 392]
Current line: [690, 211]
Current line: [690, 211, 681, 392]
Current line: [721, 394]
Current line: [721, 394, 731, 214]
Current line: [768, 211]
Current line: [768, 211, 762, 394]
Current line: [804, 396]
Current line: [804, 396, 807, 213]
Current line: [845, 213]
Current line: [845, 213, 849, 391]
Current line: [892, 395]
Current line: [892, 395, 886, 210]
Current line: [930, 370]
Current line: [930, 370, 520, 365]
Current line: [524, 342]
Current line: [524, 342, 927, 342]
Current line: [927, 320]
Current line: [927, 320, 526, 319]
Current line: [926, 297]
Current line: [926, 297, 534, 295]
Current line: [537, 271]
Current line: [537, 271, 921, 276]
Current line: [929, 251]
Current line: [929, 251, 540, 253]
Current line: [541, 225]
Current line: [541, 225, 919, 229]
Corners [[[558.3848  365.4681 ]]

 [[560

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread (0x8fdf530).
Cannot move to target thread (0x92237e0)

QObject::moveToThread: Current thread (0x92237e0) is not the object's thread

Current line: [242, 478]
Current line: [242, 478, 765, 151]
Removing last line
Current line: [766, 152]
Current line: [766, 152, 749, 279]
Current line: [780, 281]
Current line: [780, 281, 796, 154]
Current line: [824, 151]
Current line: [824, 151, 815, 277]
Current line: [847, 282]
Current line: [847, 282, 855, 155]
Current line: [884, 155]
Current line: [884, 155, 878, 279]
Current line: [910, 278]
Current line: [910, 278, 914, 155]
Current line: [945, 153]
Current line: [945, 153, 941, 275]
Current line: [973, 275]
Current line: [973, 275, 973, 154]
Current line: [1005, 153]
Current line: [1005, 153, 1007, 275]
Current line: [1036, 262]
Current line: [1036, 262, 719, 260]
Current line: [722, 243]
Current line: [722, 243, 1033, 244]
Current line: [1035, 231]
Current line: [1035, 231, 729, 228]
Current line: [730, 213]
Current line: [730, 213, 1034, 211]
Current line: [1027, 195]
Current line: [1027, 195, 733, 195]
Current line: [736, 182]
Current line: [736, 182, 1028, 179]
Current l

FileExistsError: [Errno 17] File exists: '/home/lukas/projekte/BallMill3D/utils/../labeled_chess_boards/images'

In [ ]:
sC.conf["camera_matrix"][1]

In [ ]:
sC.conf

# Save Calibration to YAML

In [ ]:
sC.save_to_yaml(f"{run_name}.yaml")

sC.save_to_yaml(f"{run_name}_exp.yaml")

In [ ]:
sC.load_from_yaml(f"{run_name}.yaml")

In [ ]:
sC.conf["anchor_point"], sC.name